<a href="https://colab.research.google.com/github/Cripytou/RPG_Python_Alura/blob/main/RPG_Python_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai
!pip install -q --upgrade google-generativeai

In [ ]:
import google.generativeai as genai
import os
import re
from google.colab import userdata
import time

try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = API_KEY
    print("Sistema: API Key carregada dos segredos do Colab.")
    genai.configure()

except Exception as e:
    print(f"Sistema: Erro ao configurar a API Key: {e}")
    print("Sistema: Por favor, verifique:")
    print("1. Você está rodando no Google Colab?")
    print("2. Você salvou sua API Key nos Segredos do Colab com o nome 'GOOGLE_API_KEY'?")
    print("3. A biblioteca 'google-colab' está instalada (`!pip install google-colab`)?")
    exit()

MODEL_NAME = 'gemini-1.5-flash'
model = None

try:
    print(f"Sistema: Tentando carregar o modelo '{MODEL_NAME}'...")
    model = genai.GenerativeModel(MODEL_NAME)
    print(f"Sistema: Modelo '{MODEL_NAME}' carregado com sucesso.")
except Exception as e:
    print(f"Sistema: Erro ao carregar o modelo '{MODEL_NAME}': {e}")
    print("Sistema: Por favor, verifique:")
    print("1. Sua conexão com a internet.")
    print(f"2. O nome do modelo ('{MODEL_NAME}') está correto e disponível para sua conta na Google AI Studio/Vertex AI?")
    print("3. A biblioteca google-generativeai está atualizada (`!pip install --upgrade google-generativeai` no Colab).")

if model is None:
    print("\nSistema: Falha crítica: Não foi possível inicializar o modelo Gemini. Encerrando o script.")
    exit()

player_hp = 10
player_level = 1
max_hp = 10
game_phase = "school"
player_name = ""

ART = {
    "school": r"""
     _____
    /     \
   /       \
  / PYTHON  \
 /  SCHOOL   \
/_____________\
    |     |
    |     |
    |_____|
""",
    "forest": r"""
   /\
  /  \  /\
 /    \/  \
/          \
|  FOREST    |
 \          /
  \ /\/\   /
   \/    \/
""",
    "castle": r"""
     /\
    /  \
   /____\
   |    |
   |    |
   |    |
  /|    |\
 / |    | \
/__|____|__\
""",
    "enemies": {
        "SyntaxError": r"""
 ( o.o )
>(  ^  )<
  /    \
""",
        "TypeError": r"""
 ( x.x )
=(  -  )=
  /    \
""",
        "NameError": r"""
 (?.?)
<(=Y=)>
  / \
""",
        "MagoSupremo": r"""
  ( ^_^ ) ?
 <(  O  )>|
  /|__|\  |
   /  \   |
"""
    }
}

system_instruction = f"""
Você é o Mestre de Jogo (GM) de um RPG de texto solo chamado "A Ascensão do Coder". Seu objetivo principal é ensinar conceitos básicos de Python de forma divertida e simplificada
pois o jogo é para iniciantes em programação, tudo isso através da narrativa e de desafio.

Quando for narrar, seja breve e não se enrole para explicar o contexto do mundo, mesmo sendo um RPG, ainda é um jogo focado para aprendizagem de Python.

Não esqueça que seu objetivo é ensinar python, sempre explique, além de Mestre do Jogo, você é um professor.

Comece SEMPRE o primeiro desafio como um código muito fácio e básico, apenas para iniciar o jogador.

O Cenário: Um mundo de fantasia onde Python é a força mágica fundamental. O Mago Supremo Corruptus usa "Erros" de código perversos para distorcer a realidade e trazer o caos.
O Jogador: Um jovem aprendiz de mago do código, começando no Nível {player_level} com {player_hp}/{max_hp} HP. Ele está iniciando sua jornada na Escola de Magia de Python. Sua classe é Mago do Código.
Fase Atual do Jogo: {game_phase.capitalize()}.

Objetivo do jogador: Aprender magia (Python) resolvendo desafios para se tornar poderoso o suficiente para deter Corruptus.

Progressão:
De 2 desafios na fase school, 3 desafios na fase adventure, e 2 desafio na fase wizard.
Após terminar uma área, você mestre vai avançar o jogador para a próxima área.
Na última fase, o jogador deve faser um desafio para abrir a porta do castelo e enfrentar o Mago Supremo.

Progresso da Aventura (Curta - 3 Fases/Níveis):
1.  Escola de Magia (Fase 'School', Nível 1, 10 HP Máx, Cura por magia: 2 HP): Foco em aprender conceitos BÁSICOS (variáveis, tipos, print/input, operações básicas) através de desafios teóricos/mágicos. SEM combate com dano real por falha.
2.  Aventura no Mundo (Fase 'Adventure', Nível 2, 15 HP Máx, Cura por magia: 3 HP): O jogador explora, enfrenta "Erros" (inimigos como SyntaxError, TypeError, NameError). Combate = resolver DESAFIOS PYTHON práticos. Tomar dano por falha é possível. Introduzir condicionais, loops, listas, funções simples.
3.  Confronto Final (Fase 'Wizard', Nível 3, 20 HP Máx, Cura por magia: 5 HP): O jogador alcança a fortaleza de Corruptus. A batalha final é um desafio Python complexo que exige a combinação de conceitos aprendidos. O Mago Supremo é o inimigo final.

Mecânicas do Jogo:
-   **Interação com Desafios Python:** Quando você apresenta um desafio (seja um exercício na escola ou um "Erro" inimigo), o jogador responderá digitando o CÓDIGO PYTHON que ele acredita resolver o desafio, diretamente no input do jogo. Seu script Python me enviará este texto como parte da ação do jogador.
    * **Avaliação do Código:** Sua tarefa é LER o CÓDIGO PYTHON que o jogador enviar (como texto) e avaliar se ele está CORRETO ou atinge o objetivo do desafio que você propôs. Não se preocupe em "executar" o código; você vai analisá-lo textualmente com base no seu conhecimento de Python e no contexto do desafio. Ao avaliar, mencione explicitamente se o código parece "Correto" ou "Incorreto" na sua resposta para que o jogador saiba o resultado do seu "feitiço". Use blocos de código formatados (```python\\ncode here\\n```) na sua resposta se for mostrar o código correto ou um exemplo.
    * **Sucesso:** Se o código enviado pelo jogador parecer correto e apropriado para o desafio, narre o sucesso (o Erro é dissipado, o caminho se abre, o exercício foi completado, etc.).
    * **Falha:** Se o código parecer incorreto, incompleto ou não resolver o desafio, narre o fracasso.
-   **Combate (Contra 'Erros' nas fases 'Adventure' e 'Wizard'):** Falhar em resolver o desafio Python (combate) resulta em dano!
    * Se o jogador falhar no desafio de um Erro, narre o ataque do Erro e QUANTO dano o jogador toma. Para indicar o dano que o script deve processar, você **DEVE** usar OBRIGATORIAMENTE a seguinte frase em algum lugar da sua resposta, exatamente como mostrado: "DANO: <quantidade_de_dano> HP!". Exemplo: "O TypeError contra-ataca com uma onda de incompatibilidade! DANO: 4 HP!". O script do jogo irá ler esta frase e aplicar o dano automaticamente. NÃO peça mais ao jogador para usar REPORT_HP para dano sofrido.
    * Na fase 'School', falhar em um desafio não causa dano, apenas feedback didático e a necessidade de tentar novamente.
-   **Cura (Magia):** O jogador pode tentar usar magia de cura (digitando "curar", "magia de cura", etc.). Isso requer que ele "canalize energia" (escreva ou descreva o código Python para somar vida).
    * Quando o jogador disser que quer curar, você narra o processo (ligando-o a um conceito Python, talvez somar a uma variável de HP). PEÇA a ele para usar seu código Python para calcular o novo HP, informando a quantidade de cura baseada no Nível atual (Nível 1: 2 HP, Nível 2: 3 HP, Nível 3: 5 HP). APÓS isso, instrua o jogador a reportar seu novo HP usando o comando: "Use o comando `REPORT_HP <seu_novo_hp>` na sua próxima linha para atualizar seu HP.". (A cura continua manual via REPORT_HP).
    * NÃO existem itens de cura físicos.
-   **Nível e HP Máximo:** O jogador começa no Nível 1 (10 HP Máx).
    * Ao narrar a transição da fase 'school' para 'adventure', você deve mencionar que o jogador atingiu o Nível 2 e que seu HP Máximo aumentou para 15.
    * Ao narrar a transição da fase 'adventure' para 'wizard', você deve mencionar que o jogador atingiu o Nível 3 e que seu HP Máximo aumentou para 20.
    * O script Python gerencia a atualização real desses números, mas sua narrativa valida e comunica ao jogador.
-   **HP Atual:** O HP atual do jogador é gerenciado pelo script Python. O script irá atualizar o HP automaticamente quando você usar a frase "DANO: <quantidade_de_dano> HP!". Para cura, o jogador usará `REPORT_HP` conforme instruído. Em cada turno, o script incluirá o HP atual, Nível e Fase do jogador no início da mensagem para seu contexto.
-   **Desafios Python:** Crie desafios CLAROS e relevantes para a narrativa de cada fase, focando nos conceitos apropriados para o nível do jogador. APRESENTE o desafio como um trecho de código para corrigir/completar ou um problema que requer código como solução. Adapte a complexidade para School (básico), Adventure (intermediário), Wizard (combinação). Use nomes de Erros (SyntaxError, TypeError, NameError, etc.) para inimigos na fase 'Adventure' e 'Wizard'.

Sua Tarefa Como GM:
-   Descreva o cenário da Fase Atual de forma imersiva e mágica.
-   Apresente desafios de Python CLAROS e ligados à história/Erros, indicando que a resposta esperada é o CÓDIGO PYTHON.
-   **Leia e avalie o código enviado pelo jogador (como texto). Seja explícito se o código está Correto ou Incorreto na sua avaliação.** Use blocos de código formatados na sua resposta quando for apropriado (ex: ```python\\nseu_codigo\\n```).
-   Narrem o sucesso/falha da resolução do desafio de forma envolvente.
-   Narrem dano usando a frase "DANO: <quantidade_de_dano> HP!" e para cura, instrua o jogador a usar `REPORT_HP <seu_novo_hp>` logo após ele calcular o valor.
-   Narrem as transições de fase e nível nos momentos apropriados (quando a narrativa sugere que o jogador completou uma fase).
-   Reaja às ações do jogador mantendo a coerência com as regras e a história.
-   Mantenha o foco em ensinar Python através da aventura.

**Comece agora, descrevendo o cenário inicial na Escola de Magia de Python. Apresente o jogador (um jovem aprendiz) e lance o primeiro desafio básico de Python que requer código como resposta para começar a aventura.**
"""

try:
    chat = model.start_chat(history=[
        {"role": "user", "parts": system_instruction},
    ])
    print("\nSistema: Chat com Gemini iniciado.")
except Exception as e:
    print(f"Sistema: Ocorreu um erro ao iniciar o chat com a IA: {e}")
    print("Sistema: Encerrando o script.")
    exit()

print("\n--- INÍCIO DA AVENTURA ---")
print(ART["school"])
print(f"Sistema: Conectado ao Mestre ({MODEL_NAME})...")

player_name = input("Sistema: Qual o nome do herói? ")
print(f"Sistema: Bem-vindo, {player_name}!")

initial_player_action = f"Nome do Jogador: {player_name}. HP: {player_hp}/{max_hp}, Nível: {player_level}, Fase: {game_phase.capitalize()}. Minha ação: Estou pronto para minha primeira lição na Escola Python. O que acontece agora?"

try:
    initial_response = chat.send_message(initial_player_action)
    print("\nMestre:", initial_response.text)
except Exception as e:
    print(f"Sistema: Ocorreu um erro ao obter a primeira resposta da IA: {e}")
    print("Sistema: Encerrando o script.")
    exit()

while True:
    print("-" * 30)
    print(f"Seu Status -> Nome: {player_name} | HP: {player_hp}/{max_hp} | Nível: {player_level} | Fase: {game_phase.capitalize()}")
    player_input = input("Sua ação ou código Python (ou 'sair' para terminar, 'status' para ver estado): ")
    print("-" * 30)

    if player_input.lower() == 'sair':
        print("Sistema: Fim da aventura! O código do mundo permanece ameaçado... por enquanto.")
        break

    if player_input.lower() == 'status':
        print(f"Sistema: Status Atual: Nome={player_name}, HP={player_hp}/{max_hp}, Nível={player_level}, Fase={game_phase.capitalize()}")
        continue

    report_hp_match = re.match(r'^REPORT_HP\s+(\d+)$', player_input, re.IGNORECASE)
    if report_hp_match:
        reported_hp = int(report_hp_match.group(1))
        old_hp = player_hp
        player_hp = min(reported_hp, max_hp)
        player_hp = max(0, player_hp)
        print(f"(Sistema: Seu HP foi atualizado de {old_hp} para {player_hp}/{max_hp} após a cura)")

        if player_hp <= 0:
            print("\nSistema: Seu HP chegou a zero.")
            print("Sistema: GAME OVER!")
            break
        player_input = input("Sistema: HP atualizado pela cura. Qual sua *próxima* ação ou código Python? ")
        if player_input.lower() == 'sair':
            print("Sistema: Fim da aventura!")
            break
        if player_input.lower() == 'status':
            print(f"Sistema: Status Atual: Nome={player_name}, HP={player_hp}/{max_hp}, Nível={player_level}, Fase={game_phase.capitalize()}")
            continue

    if game_phase == "school" and player_level == 1 and any(phrase in player_input.lower() for phrase in ["sair da escola", "deixo a escola", "saio para aventura", "ir para floresta"]):
        game_phase = "adventure"
        player_level = 2
        max_hp = 15
        player_hp = max_hp
        print("\n*** Sistema: Sua maestria básica em Python desbloqueou novos caminhos! VOCÊ ATINGIU O NÍVEL 2! ***")
        print(f"*** Sistema: Sua energia vital (HP Máximo) aumentou para {max_hp}! ***")
        print(ART["forest"])
    elif game_phase == "adventure" and player_level == 2 and any(phrase in player_input.lower() for phrase in ["chego no mago supremo", "entro na fortaleza", "enfrento corruptus", "ir para castelo"]):
        game_phase = "wizard"
        player_level = 3
        max_hp = 20
        player_hp = max_hp
        print("\n*** Sistema: Você alcança o covil de Corruptus! Sente o poder do código fluindo! VOCÊ ATINGIU O NÍVEL 3! ***")
        print(f"*** Sistema: Sua energia vital (HP Máximo) final aumentou para {max_hp}! Prepare-se! ***")
        print(ART["castle"])

    contextualized_input = f"Nome do Jogador: {player_name}. HP: {player_hp}/{max_hp}, Nível: {player_level}, Fase: {game_phase.capitalize()}. Minha ação (Pode conter código): {player_input}"

    try:
        response = chat.send_message(contextualized_input)
        response_text = response.text

        damage_match = re.search(r"DANO:\s*(\d+)\s*HP!", response_text, re.IGNORECASE)
        if damage_match:
            damage_taken = int(damage_match.group(1))
            old_hp_before_damage = player_hp
            player_hp -= damage_taken
            player_hp = max(0, player_hp)
            print(f"\n(Sistema: Você sofreu {damage_taken} de dano automático! Seu HP foi de {old_hp_before_damage} para {player_hp}/{max_hp})")

            if player_hp <= 0:
                print("\nMestre:", response_text)
                print("\nSistema: Seu HP chegou a zero. As forças do Erro o dominaram.")
                print("Sistema: GAME OVER!")
                break

        displayed_art = False
        for enemy_name, enemy_art in ART["enemies"].items():
            if re.search(r'\b' + re.escape(enemy_name) + r'\b', response_text, re.IGNORECASE):
                print(f"\n--- {enemy_name} ---")
                print(enemy_art)
                displayed_art = True
                break

        print("\nMestre:", response_text)

        if any(phrase in response_text for phrase in ["O Mago Supremo Corruptus cai!", "O mundo está salvo!", "Você derrotou Corruptus!", "Corruptus foi derrotado"]):
            print("\n*** Sistema: PARABÉNS! VOCÊ DERROTOU O MAGO SUPREMO E SALVOU O MUNDO COM SEU DOMÍNIO DO PYTHON! ***")
            break

        if player_hp <= 0 and not damage_match:
            print("\nSistema: Seu HP chegou a zero. As forças do Erro o dominaram.")
            print("Sistema: GAME OVER!")
            break

    except Exception as e:
        print(f"Sistema: Ocorreu um erro durante a interação com a IA: {e}")
        print("Sistema: Tentando continuar... ou digite 'sair' para encerrar.")